In [ ]:
#!pip install pandas
#!pip install xlrd==1.2.0
#!pip install xlutils
#!pip install import_ipynb
import import_ipynb
#import CO_Calc
import pandas as pd
import os

In [ ]:
def write_co_attainment(sub_name,attainment):
    excel_name = sub_name.replace('/',' ')+'.xls'
    df_Paper = pd.read_excel(excel_name)
    total_length = len(df_Paper.loc[:,])
    internal = []
    internal_att = []
    univ = []
    univ_att =[]
    for i in df_Paper.iloc[:,1]:
        internal.append(i)
    for i in df_Paper.iloc[:,2]:
        internal_att.append(i)
        univ.append(attainment)
        if attainment >= 60:
            univ_att.append(3)
        elif attainment >= 50:
            univ_att.append(2)
        elif attainment >= 40:
            univ_att.append(1)
        else:
            univ_att.append(0)

    from xlrd import open_workbook
    from xlutils.copy import copy
    rb = open_workbook(excel_name)
    wb = copy(rb)
    s = wb.get_sheet(0)
    for i in range(total_length):
        if attainment >= 60:
            s.write(i+1,4,3)
        elif attainment >= 50:
            s.write(i+1,4,2)
        elif attainment >= 40:
            s.write(i+1,4,1)
        else:
            s.write(i+1,4,0)
        s.write(i+1,3,attainment)
    for i in range(total_length):
        s.write(i+1,5,float(str((internal[i]*0.5)+(univ[i]*0.5))[:5]))
    for i in range(total_length):
        s.write(i+1,6,float(str((internal_att[i]*0.5)+(univ_att[i]*0.5))[:5]))
    wb.save(excel_name)
    

In [ ]:
def get_sub_marks(excel_file_name):
    regulation = int(input("\nEnter regulation 2017 or 2013 for "+str(excel_file_name)+"...\n"))
    df_Paper = pd.read_excel(excel_file_name)

    column_names = df_Paper.columns[:]    
    counter = 0
    for col_name in column_names:
        grades = []
        calc = 0
        breakable=0
        for i in df_Paper.loc[1:,col_name]:
            breakable+=1
            if pd.isna(i) or i == '0' or i==0 or i==0.0 or (col_name!='S.NO' and counter == breakable):
                sub_name = col_name
                if col_name in column_names[3:]:
                    more_than_60 = 0
                    if regulation == 2017:
                        for i in grades:
                            temp = str(i).strip()
                            if temp in "AA+BB+O":
                                more_than_60+=1
                    else:
                        for i in grades:
                            temp = str(i).strip()
                            if temp in "SABCD":
                                more_than_60+=1   
            
                    write_co_attainment(sub_name, more_than_60 / (counter-1) * 100)
                break
            if col_name in column_names[3:]:
                grades.append(i)
            if col_name == 'S.NO':
                counter+=1

In [ ]:
myList = []
for i in os.listdir():
    if "UNIV" in i:
        print(i)
        get_sub_marks(i)
        print("\nCalculating university marks for "+str(i)+" ...\n")